### Earth Genome uses the ESA tile system with a 5-digit identifier. 

### In [00_s2_tile_management_sea.ipynb](./00_s2_tile_management_sea.ipynb) we've already aggregated tile IDs to country boundaries 

### we'll use that JSON file to help us pull only the EG parquet files we need for a country..

### Downloading Earthgenome Geoparquet's

In [ ]:
# let's check how EG organized their data in the aws location
!aws s3 ls s3://earthgenome/earthindexembeddings/2024/ --endpoint-url=https://data.source.coop


Read in our country-tile JSON reference

In [20]:
import os
import json
import geopandas as gpd
tile_dict = json.load(open("../esa_grid/adm0_tiles_by_country_SEA.json"))
print(tile_dict.keys())

dict_keys(['Cambodia', 'Indonesia', "Lao People's Democratic Republic", 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'])


In [21]:
country = "Cambodia"
tiles = tile_dict[country]
tiles.sort()
print(f"{len(tiles)} S2 tiles covering {country}")
for t in tiles:
    print(t)

36 S2 tiles covering Cambodia
47PRP
47PRQ
47PRR
48PTA
48PTR
48PTS
48PTT
48PTU
48PTV
48PUA
48PUB
48PUS
48PUT
48PUU
48PUV
48PVA
48PVB
48PVS
48PVT
48PVU
48PVV
48PWA
48PWS
48PWT
48PWU
48PWV
48PXA
48PXB
48PXS
48PXT
48PXU
48PXV
48PYA
48PYB
48PYU
48PYV


In [ ]:
import subprocess

dryrun=True

for t in tiles:
    suffix = "2024-01-01_2025-01-01.parquet"
    pattern = f"s3://earthgenome/earthindexembeddings/2024/{t}_{suffix}"
    cmd = f"aws s3 cp {pattern} ../embeddings/earthgenome/2024/{t}_{suffix} --endpoint-url=https://data.source.coop"
    if dryrun:

        print(cmd)
    else:
        print(f"Running {cmd}")
        try:
            subprocess.run(cmd, shell=True, capture_output=True, check=True)
        except subprocess.CalledProcessError as e:
            print(f"Error copying {t}: {e}")
            # If the file does not exist, we can skip it
            if "does not exist" in e.stderr.decode():
                print(f"File {t} does not exist, skipping.")
                continue
            else:
                raise
    # break


aws s3 cp s3://earthgenome/earthindexembeddings/2024/47PRP_2024-01-01_2025-01-01.parquet ../embeddings/earthgenome/2024/47PRP_2024-01-01_2025-01-01.parquet --endpoint-url=https://data.source.coop
aws s3 cp s3://earthgenome/earthindexembeddings/2024/47PRQ_2024-01-01_2025-01-01.parquet ../embeddings/earthgenome/2024/47PRQ_2024-01-01_2025-01-01.parquet --endpoint-url=https://data.source.coop
aws s3 cp s3://earthgenome/earthindexembeddings/2024/47PRR_2024-01-01_2025-01-01.parquet ../embeddings/earthgenome/2024/47PRR_2024-01-01_2025-01-01.parquet --endpoint-url=https://data.source.coop
aws s3 cp s3://earthgenome/earthindexembeddings/2024/48PTA_2024-01-01_2025-01-01.parquet ../embeddings/earthgenome/2024/48PTA_2024-01-01_2025-01-01.parquet --endpoint-url=https://data.source.coop
aws s3 cp s3://earthgenome/earthindexembeddings/2024/48PTR_2024-01-01_2025-01-01.parquet ../embeddings/earthgenome/2024/48PTR_2024-01-01_2025-01-01.parquet --endpoint-url=https://data.source.coop
aws s3 cp s3://earth

Look at a geoparquet file

In [ ]:
# look at one
files = os.listdir("../embeddings/earthgenome/2024")
print(f"{len(list(files))} files:\n {files}")
file = os.path.join("../embeddings/earthgenome/2024", files[0])
print(file)
df = gpd.read_parquet(file)
print(df.head())


36 files:
 ['48PVS_2024-01-01_2025-01-01.parquet', '48PYB_2024-01-01_2025-01-01.parquet', '48PXS_2024-01-01_2025-01-01.parquet', '48PUV_2024-01-01_2025-01-01.parquet', '48PXB_2024-01-01_2025-01-01.parquet', '48PVU_2024-01-01_2025-01-01.parquet', '48PYV_2024-01-01_2025-01-01.parquet', '48PWU_2024-01-01_2025-01-01.parquet', '48PWS_2024-01-01_2025-01-01.parquet', '48PUT_2024-01-01_2025-01-01.parquet', '48PUB_2024-01-01_2025-01-01.parquet', '48PTT_2024-01-01_2025-01-01.parquet', '48PUS_2024-01-01_2025-01-01.parquet', '47PRP_2024-01-01_2025-01-01.parquet', '48PYU_2024-01-01_2025-01-01.parquet', '48PYA_2024-01-01_2025-01-01.parquet', '48PWA_2024-01-01_2025-01-01.parquet', '48PVB_2024-01-01_2025-01-01.parquet', '48PXA_2024-01-01_2025-01-01.parquet', '48PVT_2024-01-01_2025-01-01.parquet', '48PVV_2024-01-01_2025-01-01.parquet', '48PXV_2024-01-01_2025-01-01.parquet', '48PTR_2024-01-01_2025-01-01.parquet', '48PUA_2024-01-01_2025-01-01.parquet', '47PRQ_2024-01-01_2025-01-01.parquet', '48PWV_2024-0

we'll add a tile column to help us stay organized

In [ ]:
# overwrite all files to add tile column
for file in files:
    file_path = os.path.join("../embeddings/earthgenome/2024", file)
    df = gpd.read_parquet(file_path)
    df.loc[:,'tile'] = os.path.basename(file).split("_")[0]
    df.to_parquet(file_path, index=False)
    print(f"Updated {file} with tile column.")


Updated 48PVS_2024-01-01_2025-01-01.parquet with tile column.


In [66]:
print(gpd.read_parquet(file_path).head())

                  id                                          embedding  \
0  31646144126408287  [4.21372, -0.47599944, 2.2734528, 1.9703406, 1...   
1  31646144130651742  [4.2551923, -0.7167304, 2.658962, 0.3453492, 1...   
2  31646144176805470  [4.234766, 0.004074624, 2.8566103, 0.62115043,...   
3  31646144181179995  [4.007387, -0.4687737, 2.4155247, 0.657522, 1....   
4  31646144193779291  [4.0800915, -0.44566143, 2.454309, 0.83217186,...   

                    geometry   tile  
0  POINT (104.09029 9.86266)  48PVS  
1   POINT (104.0932 9.86266)  48PVS  
2  POINT (104.09612 9.86266)  48PVS  
3  POINT (104.09904 9.86266)  48PVS  
4  POINT (104.10196 9.86266)  48PVS  


### Loading Data into BigQuery

In [67]:
# create new bucket
BUCKET = "gs://embeddings-sea"
PROJECT_ID = "g4g-eaas"
LOCATION = "us-central1"

In [ ]:
# upload parquet files to gcs
# try gcloud storage sync..
gcloud_folder = f"{BUCKET}/earthgenome/2024"
!gcloud storage rsync ../embeddings/earthgenome/2024 $gcloud_folder \
    --project=$PROJECT_ID

At file://../embeddings/earthgenome/2024/*, worker process 3209627 thread 135100769339200 listed 36...
At gs://embeddings-sea/embeddings/earthgenome/2024/*, worker process 3209627 thread 135100769339200 listed 1...
uploading large objects. If you would like to opt-out and instead
perform a normal upload, run:
`gcloud config set storage/parallel_composite_upload_enabled False`
If you would like to disable this warning, run:
`gcloud config set storage/parallel_composite_upload_enabled True`
Note that with parallel composite uploads, your object might be
uploaded as a composite object
(https://cloud.google.com/storage/docs/composite-objects), which means
that any user who downloads your object will need to use crc32c
checksums to verify data integrity. gcloud storage is capable of
computing crc32c checksums, but this might pose a problem for other
clients.

Copying file://../embeddings/earthgenome/2024/47PRP_2024-01-01_2025-01-01.parquet to gs://embeddings-sea/embeddings/earthgenome/2024/

In [70]:
# load geoparquet file from gcs to bigquery
DATASET_ID = "embeddings_sea"
TABLE_ID = "earthgenome_cambodia"
FULL_TABLE = f"{PROJECT_ID}:{DATASET_ID}.{TABLE_ID}"
FOLDER = "earthgenome/2024"
print(FULL_TABLE)
for file in files:
    URI = f"{BUCKET}/{FOLDER}/{file}"

    print(URI)
    !bq --location=$LOCATION --project_id=$PROJECT_ID \
            load \
                --source_format=PARQUET \
                $FULL_TABLE \
                $URI

g4g-eaas:embeddings_sea.earthgenome_cambodia
gs://embeddings-sea/earthgenome/2024/48PVS_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r242c47c8c8509611_00000197319a72b8_1 ... (36s) Current status: DONE   
gs://embeddings-sea/earthgenome/2024/48PYB_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r72c0e1a5c9f5091d_00000197319b1505_1 ... (36s) Current status: DONE   
gs://embeddings-sea/earthgenome/2024/48PXS_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r343feb9fd97f1f9b_00000197319bb1b2_1 ... (35s) Current status: DONE   
gs://embeddings-sea/earthgenome/2024/48PUV_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r43aaab8565ba13f1_00000197319c4c2f_1 ... (35s) Current status: DONE   
gs://embeddings-sea/earthgenome/2024/48PXB_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r421595f151593996_00000197319ce7c0_1 ... (35s) Current status: DONE   
gs://embeddings-sea/earthgenome/2024/48PVU_2024-01-01_2025-01-01.parquet
Waiting on bqjob_r45897fa0f5e5028d_00000197319d8234_1 ... (24s) Current statu

### Minor transforms of the BQ table 

n do a small post-processing query to get the embedding field converted correctly for vector search..

vector search indexing requires the embedding field to be of type `ARRAY<FLOAT>`

the load operation turns 'embedding' field into a double-nested STRUCT data type, innermost child containing list of floats.. 

so we have to unpack that list from the nested structure, final data type being `ARRAY<FLOAT64>`

In [ ]:
from google.cloud import bigquery
query = f"""
SELECT
  eg.id,
  eg.tile,
  ST_GEOGFROMTEXT(grouped.geometry_text) AS geometry,
  ARRAY_AGG(e.element) AS embedding
FROM
  `{PROJECT_ID}`.`{DATASET_ID}`.`{TABLE_ID}` AS eg
CROSS JOIN
  UNNEST(eg.embedding.list) AS e
JOIN (
  SELECT id, tile, ST_ASTEXT(geometry) AS geometry_text
  FROM `{PROJECT_ID}`.`{DATASET_ID}`.`{TABLE_ID}`
  GROUP BY id, tile, geometry_text
) AS grouped ON eg.id = grouped.id AND eg.tile = grouped.tile AND ST_ASTEXT(eg.geometry) = grouped.geometry_text
GROUP BY eg.id, eg.tile, grouped.geometry_text
"""

# Run the query and save the result to a new table
result_table = f"{PROJECT_ID}.{DATASET_ID}.earthgenome_cambodia_v1"
job_config = bigquery.QueryJobConfig(destination=result_table)
client = bigquery.Client()
job = client.query(query, job_config=job_config)
job.result()  # Wait for the job to complete

In [73]:
# Check if the result_table exists

def table_exists(client, table_id):
    try:
        client.get_table(table_id)
        print(f"Table {table_id} exists.")
        return True
    except Exception as e:
        print(f"Table {table_id} does not exist. Error: {e}")
        return False

table_exists(client, result_table)

Table g4g-eaas.embeddings_sea.earthgenome_cambodia_v1 exists.


True

In [74]:
# check the resulting table's schema and data
query = f"SELECT * FROM `{result_table}` LIMIT 10"
query_job = client.query(query)
# print schema
schema = query_job.result().schema
for field in schema:
    print(f"{field.name}: {field.field_type}")
for row in query_job:
    print(row)

id: INTEGER
tile: STRING
geometry: GEOGRAPHY
embedding: FLOAT
Row((31736946848215620, '47PRP', 'POINT(101.915459844242 12.0042160401163)', [4.323385715484619, 0.06400411576032639, 1.0368335247039795, 1.5289827585220337, 1.4563442468643188, -2.2122995853424072, 4.070167064666748, -0.6424505710601807, 3.405003547668457, 1.4932760000228882, 1.7559279203414917, -1.833122730255127, -0.7179238796234131, -0.41533586382865906, 0.10614579916000366, 0.00683737313374877, 0.3592619299888611, 0.36866050958633423, -2.5919840335845947, 1.8089570999145508, 2.2521915435791016, 0.4613773226737976, 1.5383117198944092, -3.08720064163208, -0.7681889533996582, 0.8207824230194092, 0.819365918636322, 1.1905938386917114, -1.2695367336273193, 2.3134665489196777, -1.4144809246063232, -0.4272822141647339, -1.8325029611587524, 0.38100242614746094, -0.9554205536842346, -1.8019816875457764, 0.023497002199292183, 1.0904444456100464, 0.07832003384828568, 0.4823470115661621, 0.007036820985376835, -0.48182833194732666, 

### Index BQ table to enable Vector Search

In [76]:
# test VECTOR SEARCH operations
in_table = '.'.join(result_table.split(".")[1:])
print(f'indexing {in_table} for vector search')
query = f"""
CREATE VECTOR INDEX my_index ON {in_table}(embedding)
OPTIONS(distance_type='COSINE', index_type='IVF', ivf_options='{{"num_lists": 1000}}');
"""

# Run the query to create the index
client = bigquery.Client(project=PROJECT_ID)
job = client.query(query)
job.result()  # Wait for the job to complete

indexing embeddings_sea.earthgenome_cambodia_v1 for vector search


Create a test target table of 1 record to perform vector search with

In [77]:
result_table = result_table+"_test_target"
query = f"SELECT * FROM {in_table} LIMIT 1"

job_config = bigquery.QueryJobConfig(destination=result_table)
job = client.query(query,job_config=job_config)
job.result()  # Wait for the job to complete

Run a Vector Search!

In [89]:
import datetime
target_table = '.'.join(result_table.split(".")[1:])
print(target_table)
query = f"""
SELECT query.id AS target_id,
  query.tile AS target_tile,
  base.id AS base_id,
  base.tile AS base_tile,
  distance
FROM
  VECTOR_SEARCH(
    TABLE {in_table},
    'embedding',
    TABLE {target_table},
    top_k => 11,
    distance_type => 'COSINE',
    options => '{{"fraction_lists_to_search": 0.005}}')
ORDER BY distance
LIMIT 10
OFFSET 1;
"""

# Run the query to create the index
client = bigquery.Client(project=PROJECT_ID)
search_result_table = f"{PROJECT_ID}.{DATASET_ID}.vector_search_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
job_config = bigquery.QueryJobConfig(destination=search_result_table)
job = client.query(query,job_config=job_config)
job.result()  # Wait for the job to complete

embeddings_sea.earthgenome_cambodia_v1_test_target


In [90]:
query = f"SELECT * FROM `{search_result_table}` LIMIT 10"
query_job = client.query(query)
# print schema
schema = query_job.result().schema
for field in schema:
    print(f"{field.name}: {field.field_type}")
for row in query_job:
    print(row)

target_id: INTEGER
target_tile: STRING
base_id: INTEGER
base_tile: STRING
distance: FLOAT
Row((31737298129880965, '47PRP', 31737295412923663, '47PRP', 0.018239032724844506), {'target_id': 0, 'target_tile': 1, 'base_id': 2, 'base_tile': 3, 'distance': 4})
Row((31737298129880965, '47PRP', 31737318005760655, '47PRP', 0.018310001948530275), {'target_id': 0, 'target_tile': 1, 'base_id': 2, 'base_tile': 3, 'distance': 4})
Row((31737298129880965, '47PRP', 31737318187192539, '47PRP', 0.018820251294408008), {'target_id': 0, 'target_tile': 1, 'base_id': 2, 'base_tile': 3, 'distance': 4})
Row((31737298129880965, '47PRP', 31738782341388707, '47PRQ', 0.019642342550201808), {'target_id': 0, 'target_tile': 1, 'base_id': 2, 'base_tile': 3, 'distance': 4})
Row((31737298129880965, '47PRP', 31738794989829102, '47PRQ', 0.019749958040534477), {'target_id': 0, 'target_tile': 1, 'base_id': 2, 'base_tile': 3, 'distance': 4})
Row((31737298129880965, '47PRP', 31737295417137182, '47PRP', 0.020084470457225834), {